In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install -U transformers datasets evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
# Force upgrade key libraries
!pip install --upgrade --quiet transformers datasets evaluate --no-cache-dir


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 241.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6

In [3]:
from transformers import Trainer, TrainingArguments
print("✅  transformers & torch import cleanly!")


✅  transformers & torch import cleanly!


In [5]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer
)
import numpy as np
import evaluate

# Load uploaded dataset
df = pd.read_csv("Soft_Label_Balanced_Dataset.csv")[["prompt", "soft_label_target"]]

# Label encoding
label2id = {label: i for i, label in enumerate(df["soft_label_target"].unique())}
id2label = {v: k for k, v in label2id.items()}
df["label"] = df["soft_label_target"].map(label2id)

# Hugging Face Dataset
dataset = Dataset.from_pandas(df[["prompt", "label"]])
dataset = dataset.train_test_split(test_size=0.1, seed=42)

# Use roberta-large for stronger performance
model_name = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label2id))

# Tokenize prompts
def tokenize(example):
    return tokenizer(example["prompt"], truncation=True, padding=True, max_length=128)

tokenized = dataset.map(tokenize)

# Accuracy metric
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# Training arguments
training_args = TrainingArguments(
    output_dir="./router_classifier",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,   # Fits better on Colab GPU
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Save model
model.save_pretrained("bert_router_soft")
tokenizer.save_pretrained("bert_router_soft")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1620 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

<ipython-input-5-e389a38d4478>:57: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.101964,0.322222
2,No log,1.077411,0.333333
3,1.113000,1.057961,0.411111
4,1.113000,1.068906,0.322222
5,1.083900,1.046059,0.394444
6,1.083900,1.047155,0.383333
7,1.083900,1.033901,0.388889
8,1.078800,1.021213,0.383333
9,1.078800,1.062719,0.333333
10,1.072500,1.061356,0.333333


('bert_router_soft/tokenizer_config.json',
 'bert_router_soft/special_tokens_map.json',
 'bert_router_soft/vocab.json',
 'bert_router_soft/merges.txt',
 'bert_router_soft/added_tokens.json',
 'bert_router_soft/tokenizer.json')

In [9]:
# Replace 'your_folder_name' with the name of the folder you want to zip
!zip -r /content/newModel1.zip /content/router_classifier/checkpoint-1015

from google.colab import files
files.download('/content/newModel1.zip')

  adding: content/router_classifier/checkpoint-1015/ (stored 0%)
  adding: content/router_classifier/checkpoint-1015/model.safetensors (deflated 12%)
  adding: content/router_classifier/checkpoint-1015/rng_state.pth (deflated 25%)
  adding: content/router_classifier/checkpoint-1015/vocab.json (deflated 59%)
  adding: content/router_classifier/checkpoint-1015/optimizer.pt (deflated 21%)
  adding: content/router_classifier/checkpoint-1015/tokenizer_config.json (deflated 75%)
  adding: content/router_classifier/checkpoint-1015/config.json (deflated 54%)
  adding: content/router_classifier/checkpoint-1015/scheduler.pt (deflated 55%)
  adding: content/router_classifier/checkpoint-1015/merges.txt (deflated 53%)
  adding: content/router_classifier/checkpoint-1015/tokenizer.json (deflated 82%)
  adding: content/router_classifier/checkpoint-1015/trainer_state.json (deflated 69%)
  adding: content/router_classifier/checkpoint-1015/special_tokens_map.json (deflated 52%)
  adding: content/router_c

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
"""
router_soft.py
Train-time checkpoint:  bert_router_soft/   (or any path you choose)
Run:
    python router_soft.py             # local CLI test
or in Colab:
    # install once
    !pip install -q transformers torch requests
    # then just run this cell
"""

import os, requests, torch, torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

# ──────────────────────────── CONFIG ──────────────────────────────
MODEL_DIR = "/content/router_classifier/checkpoint-1015"   # ← change to your folder
os.environ["TOGETHER_API_KEY"] = "90e9961637294d237b0c969d3d339dba52ec869a7edf91af2aa092d2fb3c7664"
TOGETHER_URL = "https://api.together.xyz/v1/chat/completions"

REAL_LABELS = [
    "mistralai/mixtral-8x7b-chat",
    "Qwen/Qwen2.5-Coder-32B-Instruct",
    "mistralai/mistral-7b-chat",
]

ENDPOINTS = {           # label → Together endpoint
    "mistralai/mixtral-8x7b-chat": "mistralai/Mixtral-8x7B-Instruct-v0.1",
    "Qwen/Qwen2.5-Coder-32B-Instruct"   : "Qwen/Qwen2.5-Coder-32B-Instruct",
    "mistralai/mistral-7b-chat" : "mistralai/Mistral-7B-Instruct-v0.2",
}
# ──────────────────────────────────────────────────────────────────

def ensure_real_labels(cfg):
    """If cfg.id2label still has 'LABEL_0', overwrite with REAL_LABELS."""
    if list(cfg.id2label.values())[0].startswith("LABEL_"):
        cfg.id2label = {i: lbl for i, lbl in enumerate(REAL_LABELS)}
        cfg.label2id = {lbl: i for i, lbl in enumerate(REAL_LABELS)}
        cfg.save_pretrained(MODEL_DIR)
        print("📌 Updated model config with real label names.")
    return cfg

# load / patch config once
cfg = ensure_real_labels(AutoConfig.from_pretrained(MODEL_DIR))
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model     = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR, config=cfg)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def query_together(endpoint, prompt, max_tokens=256):
    payload = {
        "model": endpoint,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens,
        "temperature": 0.8,
        "top_p": 0.9,
    }
    r = requests.post(
        TOGETHER_URL,
        headers={"Authorization": f"Bearer {os.environ['TOGETHER_API_KEY']}",
                 "Content-Type": "application/json"},
        json=payload, timeout=60,
    )
    r.raise_for_status()
    return r.json()["choices"][0]["message"]["content"]

def route_prompt(prompt, max_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt",
                       truncation=True, padding=True, max_length=128).to(device)
    with torch.no_grad():
        probs = torch.softmax(model(**inputs).logits, dim=1)[0]
    idx = int(torch.argmax(probs))
    label = cfg.id2label[idx]       # now real label names
    endpoint = ENDPOINTS[label]

    print("\n─ Routing probabilities ─")
    for i, p in enumerate(probs):
        print(f"{cfg.id2label[i]:40} : {p:.4f}")

    llm_reply = query_together(endpoint, prompt, max_tokens)
    return (f"\n📢 Routed to: {endpoint}"
            f"\n🔢 Confidence: {probs[idx]:.3f}"
            f"\n📄 Response:\n{llm_reply.strip()}")

# CLI / quick test
if __name__ == "__main__":
    test_prompts = [
        "Write a detailed abstract for a research paper on the impact of quantum computing on modern cryptographic systems.",
        "Explain climate change in simple terms to a 10-year-old, using everyday examples.",
        "Why am I getting a KeyError in Python when using a dictionary, and how can I fix it?",
    ]
    for p in test_prompts:
        print(f"\n📝 Prompt: {p}")
        print(route_prompt(p))



📝 Prompt: Write a detailed abstract for a research paper on the impact of quantum computing on modern cryptographic systems.

─ Routing probabilities ─
mistralai/mixtral-8x7b-chat              : 0.8735
Qwen/Qwen2.5-Coder-32B-Instruct          : 0.0891
mistralai/mistral-7b-chat                : 0.0375

📢 Routed to: mistralai/Mixtral-8x7B-Instruct-v0.1
🔢 Confidence: 0.873
📄 Response:
Title: The Impact of Quantum Computing on Modern Cryptographic Systems: A Comprehensive Analysis

Abstract:

The advent of quantum computing has introduced a new paradigm in information processing, offering the potential to revolutionize various fields, including cryptography. This research paper provides an in-depth examination of the impact of quantum computing on modern cryptographic systems. 

The study begins by elucidating the fundamental principles of quantum computing, particularly quantum bits (qubits), superposition, and entanglement, which distinguish it from classical computing. The potential of

In [15]:
"""
router_soft.py
Train-time checkpoint:  bert_router_soft/   (or any path you choose)
Run:
    python router_soft.py             # local CLI test
or in Colab:
    # install once
    !pip install -q transformers torch requests
    # then just run this cell
"""

import os, requests, torch, torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

# ──────────────────────────── CONFIG ──────────────────────────────
MODEL_DIR = "/content/router_classifier/checkpoint-1015"   # ← change to your folder
os.environ["TOGETHER_API_KEY"] = "90e9961637294d237b0c969d3d339dba52ec869a7edf91af2aa092d2fb3c7664"
TOGETHER_URL = "https://api.together.xyz/v1/chat/completions"

REAL_LABELS = [
    "mistralai/mixtral-8x7b-chat",
    "Qwen/Qwen2.5-Coder-32B-Instruct",
    "mistralai/mistral-7b-chat",
]

ENDPOINTS = {           # label → Together endpoint
    "mistralai/mixtral-8x7b-chat": "mistralai/Mixtral-8x7B-Instruct-v0.1",
    "Qwen/Qwen2.5-Coder-32B-Instruct"   : "Qwen/Qwen2.5-Coder-32B-Instruct",
    "mistralai/mistral-7b-chat" : "mistralai/Mistral-7B-Instruct-v0.2",
}
# ──────────────────────────────────────────────────────────────────

def ensure_real_labels(cfg):
    """If cfg.id2label still has 'LABEL_0', overwrite with REAL_LABELS."""
    if list(cfg.id2label.values())[0].startswith("LABEL_"):
        cfg.id2label = {i: lbl for i, lbl in enumerate(REAL_LABELS)}
        cfg.label2id = {lbl: i for i, lbl in enumerate(REAL_LABELS)}
        cfg.save_pretrained(MODEL_DIR)
        print("📌 Updated model config with real label names.")
    return cfg

# load / patch config once
cfg = ensure_real_labels(AutoConfig.from_pretrained(MODEL_DIR))
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model     = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR, config=cfg)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def query_together(endpoint, prompt, max_tokens=256):
    payload = {
        "model": endpoint,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens,
        "temperature": 0.8,
        "top_p": 0.9,
    }
    r = requests.post(
        TOGETHER_URL,
        headers={"Authorization": f"Bearer {os.environ['TOGETHER_API_KEY']}",
                 "Content-Type": "application/json"},
        json=payload, timeout=60,
    )
    r.raise_for_status()
    return r.json()["choices"][0]["message"]["content"]

def route_prompt(prompt, max_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt",
                       truncation=True, padding=True, max_length=128).to(device)
    with torch.no_grad():
        probs = torch.softmax(model(**inputs).logits, dim=1)[0]
    idx = int(torch.argmax(probs))
    label = cfg.id2label[idx]       # now real label names
    endpoint = ENDPOINTS[label]

    print("\n─ Routing probabilities ─")
    for i, p in enumerate(probs):
        print(f"{cfg.id2label[i]:40} : {p:.4f}")

    llm_reply = query_together(endpoint, prompt, max_tokens)
    return (f"\n📢 Routed to: {endpoint}"
            f"\n🔢 Confidence: {probs[idx]:.3f}"
            f"\n📄 Response:\n{llm_reply.strip()}")

# CLI / quick test
if __name__ == "__main__":
    test_prompts = [
        "2222+2222222*2222=",
        "Write a literature-review section summarizing recent progress in solid-state lithium-air batteries, focusing on electrolyte stability and dendrite suppression",
        "Create a minimal Flask + SQLite snippet that exposes a /todos REST endpoint with GET and POST, plus a Dockerfile to run it.",
        "Write a Bash oneliner that replaces every foo with bar in all .txt files in the current directory, but only if the file also contains ‘baz’",
        "Explain the holographic principle in quantum gravity and outline how AdS/CFT duality supports it, using equations where helpful.",
        "Draft an executive summary for a government white paper on ‘Ethical Governance of Autonomous Weapons,’ highlighting key risks, regulatory gaps, and recommended international frameworks.",
        " Write a detailed abstract for a research paper on the impact of quantum computing on modern cryptographic systems."
    ]
    for p in test_prompts:
        print(f"\n📝 Prompt: {p}")
        print(route_prompt(p))



📝 Prompt: 2222+2222222*2222=

─ Routing probabilities ─
mistralai/mixtral-8x7b-chat              : 0.3430
Qwen/Qwen2.5-Coder-32B-Instruct          : 0.4271
mistralai/mistral-7b-chat                : 0.2299

📢 Routed to: Qwen/Qwen2.5-Coder-32B-Instruct
🔢 Confidence: 0.427
📄 Response:
To solve the expression \(2222 + 2222222 \times 2222\), we need to follow the order of operations, often remembered by the acronym PEMDAS (Parentheses, Exponents, Multiplication and Division (from left to right), Addition and Subtraction (from left to right)).

First, perform the multiplication:

\[ 2222222 \times 2222 = 4938271684 \]

Then, add the result to 2222:

\[ 2222 + 4938271684 = 4938273906 \]

So, the result of the expression is:

\[ 4938273906 \]

📝 Prompt: Write a literature-review section summarizing recent progress in solid-state lithium-air batteries, focusing on electrolyte stability and dendrite suppression

─ Routing probabilities ─
mistralai/mixtral-8x7b-chat              : 0.3300
Qwen/Q

## 🗺️ How the Router Chooses an Expert — Quick-Reference (paste into Colab)

| **When your prompt looks like …** | **Router’s top choice** | **Why this label wins** |
|-----------------------------------|-------------------------|--------------------------|
| *Formal, academic, or journal-style prose*  <br>• Long research abstracts  <br>• Dense theoretical explanations  <br>• Policy white-paper sections | **Mixtral-8×7 B**  | Training data tied “Mixtral” to highly technical, structured writing, so the classifier fires very confidently. |
| *Code or structured how-to tasks*  <br>• Snippets (Flask, Bash, SQL)  <br>• Quick arithmetic or regex  <br>• Bullet lists & step-by-step guides | **Qwen 32-B Coder** | Former “Yi-34B” rows were code-heavy; after relabeling to Qwen, this class now owns concise coding / math prompts. |
| *Explanations & medium-length summaries*  <br>• Concept overviews (physics, policy)  <br>• Debugging walk-throughs  <br>• Executive summaries | **Mistral-7 B** | Dataset examples for Mistral leaned toward clear narrative exposition and moderate-sized code with commentary. |

### Reading the probabilities
* **> 0.90 on one class** → router is *very* sure (distinct style match).  
* **≈ 0.33 – 0.45 each** → prompt sits between classes; consider adding clearer training examples.

### Heuristics for crafting prompts
* **Need scholarly depth?** → Write it like a journal abstract → Mixtral-8×7 B.  
* **Need code snippets or quick math?** → Ask in bullet/CLI style → Qwen.  
* **Need conceptual explanation or debug help?** → Plain narrative question → Mistral-7 B.


In [20]:
# 🔁 Install dependencies once
# !pip install -q tiktoken transformers torch requests

# ✅ Import dependencies
import os, requests, tiktoken, re
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import torch.nn.functional as F

# ✅ Cost per 1K tokens ($) — adjust these if Together prices change
COST_PER_1K = {
    "mistralai/Mixtral-8x7B-Instruct-v0.1": 0.008,
    "mistralai/Mistral-7B-Instruct-v0.2" : 0.0025,
    "Qwen/Qwen2.5-Coder-32B-Instruct"    : 0.005,
}
BASELINE_MODEL = "mistralai/Mixtral-8x7B-Instruct-v0.1"
ASSUMED_REPLY_TOKENS = 200

# ✅ Tokenizer (cl100k_base covers all)
ENC = tiktoken.get_encoding("cl100k_base")
def n_tokens(text): return len(ENC.encode(text))

# ✅ Load your trained classifier
MODEL_DIR = "/content/router_classifier/checkpoint-1015"  # ← adjust if needed
REAL_LABELS = [
    "mistralai/mixtral-8x7b-chat",
    "Qwen/Qwen2.5-Coder-32B-Instruct",
    "mistralai/mistral-7b-chat"
]
ENDPOINTS = {
    "mistralai/mixtral-8x7b-chat": "mistralai/Mixtral-8x7B-Instruct-v0.1",
    "Qwen/Qwen2.5-Coder-32B-Instruct": "Qwen/Qwen2.5-Coder-32B-Instruct",
    "mistralai/mistral-7b-chat": "mistralai/Mistral-7B-Instruct-v0.2"
}
cfg = AutoConfig.from_pretrained(MODEL_DIR)
if list(cfg.id2label.values())[0].startswith("LABEL_"):
    cfg.id2label = {i: lbl for i, lbl in enumerate(REAL_LABELS)}
    cfg.label2id = {lbl: i for i, lbl in enumerate(REAL_LABELS)}
    cfg.save_pretrained(MODEL_DIR)

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR, config=cfg).eval().to("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Together API key
os.environ["TOGETHER_API_KEY"] = "90e9961637294d237b0c969d3d339dba52ec869a7edf91af2aa092d2fb3c7664"
TOGETHER_URL = "https://api.together.xyz/v1/chat/completions"

def query_together(endpoint, prompt, max_tokens=256):
    payload = {
        "model": endpoint,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens,
        "temperature": 0.8,
        "top_p": 0.9,
    }
    r = requests.post(
        TOGETHER_URL,
        headers={"Authorization": f"Bearer {os.environ['TOGETHER_API_KEY']}"},
        json=payload,
        timeout=60,
    )
    r.raise_for_status()
    return r.json()["choices"][0]["message"]["content"]

# ✅ Your router logic (Colab-safe)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def route_prompt(prompt, max_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=128).to(device)
    with torch.no_grad():
        probs = torch.softmax(model(**inputs).logits, dim=1)[0]
    idx = int(torch.argmax(probs))
    label = cfg.id2label[idx]
    endpoint = ENDPOINTS[label]
    print(f"\n📢 Routed to: {endpoint}\n🔢 Confidence: {probs[idx]:.3f}")
    print("🔍 Probabilities:")
    for i, p in enumerate(probs): print(f"   {cfg.id2label[i]:38} : {p:.4f}")
    reply = query_together(endpoint, prompt, max_tokens)
    return reply.strip(), endpoint

# ✅ Your test prompts
prompts = [
    "Write a detailed abstract for a research paper on the impact of quantum computing on modern cryptographic systems.",
    "Explain climate change in simple terms to a 10-year-old, using everyday examples.",
    "Why am I getting a KeyError in Python when using a dictionary, and how can I fix it?",
]

# ✅ Cost comparison loop
baseline_cost = 0.0
routed_cost = 0.0

for p in prompts:
    base_tokens = n_tokens(p) + ASSUMED_REPLY_TOKENS
    baseline_cost += base_tokens / 1000 * COST_PER_1K[BASELINE_MODEL]

    response, endpoint = route_prompt(p)
    total_tok = n_tokens(p) + n_tokens(response)
    routed_cost += total_tok / 1000 * COST_PER_1K[endpoint]

# ✅ Summary
print(f"\n--- Cost Summary for {len(prompts)} Prompts ---")
print(f"🧱 Baseline (all to {BASELINE_MODEL}): ${baseline_cost:.4f}")
print(f"🧠 Smart routing total               : ${routed_cost:.4f}")
savings_pct = 100 * (baseline_cost - routed_cost) / baseline_cost
print(f"💰 Estimated savings                : {savings_pct:.1f}%")



📢 Routed to: mistralai/Mixtral-8x7B-Instruct-v0.1
🔢 Confidence: 0.873
🔍 Probabilities:
   mistralai/mixtral-8x7b-chat            : 0.8735
   Qwen/Qwen2.5-Coder-32B-Instruct        : 0.0891
   mistralai/mistral-7b-chat              : 0.0375

📢 Routed to: mistralai/Mistral-7B-Instruct-v0.2
🔢 Confidence: 0.394
🔍 Probabilities:
   mistralai/mixtral-8x7b-chat            : 0.2950
   Qwen/Qwen2.5-Coder-32B-Instruct        : 0.3111
   mistralai/mistral-7b-chat              : 0.3939

📢 Routed to: Qwen/Qwen2.5-Coder-32B-Instruct
🔢 Confidence: 0.433
🔍 Probabilities:
   mistralai/mixtral-8x7b-chat            : 0.3368
   Qwen/Qwen2.5-Coder-32B-Instruct        : 0.4330
   mistralai/mistral-7b-chat              : 0.2301

--- Cost Summary for 3 Prompts ---
🧱 Baseline (all to mistralai/Mixtral-8x7B-Instruct-v0.1): $0.0053
🧠 Smart routing total               : $0.0039
💰 Estimated savings                : 25.1%
